In [ ]:
%%capture --no-stderr
%pip install --quiet -U langchain_openai langchain_core langgraph langchain[google-genai]

In [ ]:
from pprint import pprint

from langchain_core.messages import AIMessage, HumanMessage

messages = [AIMessage(content=f"So you said you were researching ocean mamals", name = "Model")]
messages.append(HumanMessage(content = f"Yes you are right", name = "Lance"))
messages.append(AIMessage(content=f"Great, what would you like to learn about.", name="Model"))
messages.append(HumanMessage(content=f"I want to learn about the best place to see Orcas in the US.", name="Lance"))

for m in messages:
    m.pretty_print()

In [ ]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("GOOGLE_API_KEY")

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

#from langchain.chat_models import init_chat_model
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.0) 

#llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [ ]:
result = llm.invoke(messages)

In [ ]:
result

In [ ]:
def multiply(a: int, b:int) -> int:
    """
        Multiply a and b
        Args:
            a: first int
            b: second int
    
    """
    return a * b

llm_with_tools = llm.bind_tools([multiply])


In [ ]:
tool_call = llm_with_tools.invoke([
    HumanMessage(content=f"What is 2 multiplied by 3", name="Lance")    
])

In [ ]:
tool_call

In [ ]:
tool_call.tool_calls

In [ ]:
from typing import TypedDict
from typing import Annotated

from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages

class MessagesState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
    

Changing MessagesState to use builtin **MessagesState**

In [ ]:
from langgraph.graph import MessagesState

class MessagesState(MessagesState):
    # Add any keys needed beyond messages, which is pre-built 
    pass

In [ ]:
# Initial state
initial_messages = [AIMessage(content="Hello! How can I assist you?", name="Model"),
                    HumanMessage(content="I'm looking for information on marine biology.", name="Lance")
                   ]

# New message to add
new_message = AIMessage(content="Sure, I can help with that. What specifically are you interested in?", name="Model")

# Test
add_messages(initial_messages , new_message)

In [ ]:
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END

def tool_calling_llm(state: MessagesState):
    return {"messages": llm_with_tools.invoke(state["messages"])}


builder = StateGraph(MessagesState)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_edge(START, "tool_calling_llm")
builder.add_edge("tool_calling_llm", END)

graph = builder.compile()

display(Image(graph.get_graph().draw_mermaid_png()))


In [ ]:
messages = graph.invoke({
    "messages": HumanMessage(content = "Hello")
})

messages

In [ ]:
messages = graph.invoke({
    "messages": HumanMessage(content = "What is 2 multiply 3")
})

messages

In [ ]:
for m in messages['messages']:
    m.pretty_print()